# BBS and Trading Toolkit Examples

This notebook demonstrates the basic usage of the BBS (Bulletin Board System) and Coinbase Trading Toolkit.

## Table of Contents
1. [BBS Basic Operations](#bbs-basic-operations)
2. [BBS Python API Examples](#bbs-python-api)
3. [Trading Toolkit Examples](#trading-examples)
4. [AI Agent Coordination Patterns](#ai-coordination)


## BBS Basic Operations

Let's start by importing the necessary modules and setting up the BBS system.

In [ ]:
# Import BBS modules
import sys
import os

# Add the project root to Python path
sys.path.append('..')

from bulletin_board import core
from bulletin_board.models import Message
from datetime import datetime

In [ ]:
# Post a message to the bulletin board
core.post(
    message="System initialized successfully",
    subject="Startup Notification",
    author="notebook-agent"
)

print("Message posted successfully!")

In [ ]:
# Read recent messages
print("Recent messages:")
recent_messages = core.tail(n=5)

for msg in recent_messages:
    print(f"[{msg.timestamp.strftime('%H:%M:%S')}] {msg.author}: {msg.subject}")
    print(f"  {msg.message}")
    print("---")

## BBS Python API Examples

Here are more advanced examples of using the BBS Python API for agent coordination.

In [ ]:
# Example: Task Queue Pattern
def post_task(task_id, task_type, task_data, priority="normal"):
    """Post a task to the queue for other agents to pick up."""
    task_message = f"Task ID: {task_id}, Type: {task_type}, Priority: {priority}"
    task_details = f"Data: {task_data}"
    
    core.post(
        message=f"{task_message}\n{task_details}",
        subject=f"TASK:{task_type}"
    )
    
def post_status_update(agent_id, status, details=""):
    """Post agent status update."""
    core.post(
        message=f"Agent {agent_id} status: {status}. {details}",
        subject=f"STATUS:{agent_id}",
        author=agent_id
    )

# Example usage
post_task("001", "data_analysis", "Process BTC price data", "high")
post_status_update("agent-analyzer", "busy", "Processing task 001")

In [ ]:
# Example: Message Filtering and Search
from datetime import datetime, timedelta

# Get tasks posted in last hour
one_hour_ago = datetime.now() - timedelta(hours=1)
recent_tasks = core.read(since=one_hour_ago)

task_messages = [msg for msg in recent_tasks if msg.subject.startswith('TASK:')]

print(f"Found {len(task_messages)} tasks in the last hour:")
for task in task_messages:
    print(f"- {task.subject}: {task.message[:50]}...")

# Get status updates
status_messages = [msg for msg in recent_tasks if msg.subject.startswith('STATUS:')]
print(f"\nFound {len(status_messages)} status updates:")
for status in status_messages:
    print(f"- {status.subject}: {status.message}")

## Trading Toolkit Examples

Demonstrate basic trading operations (requires API credentials).

In [ ]:
# Check if we can access the trading scripts
import subprocess
import json

try:
    # Try to get account balances (safe operation)
    result = subprocess.run([
        'python3', '../scripts/account_balances.py', '--json'
    ], capture_output=True, text=True, check=True)
    
    balances = json.loads(result.stdout)
    print("Account balances retrieved successfully!")
    print(f"Found {len(balances)} accounts with balances")
    
except subprocess.CalledProcessError as e:
    print(f"Could not retrieve balances: {e}")
    print("Make sure API credentials are configured")
except FileNotFoundError:
    print("Trading scripts not found - make sure you're in the right directory")

In [ ]:
# Demo the ladder tools (safe - no actual trading)
try:
    result = subprocess.run([
        'python3', '../scripts/demo_ladder_tools.py', '--quick'
    ], capture_output=True, text=True, check=True)
    
    print("Ladder tools demo output:")
    print(result.stdout)
    
except subprocess.CalledProcessError as e:
    print(f"Demo failed: {e}")
    print(result.stderr if 'result' in locals() else "No error details")
except FileNotFoundError:
    print("Demo script not found")

## AI Agent Coordination Patterns

Best practices for coordinating multiple AI agents using the BBS system.

In [ ]:
# Pattern 1: Distributed Task Processing
class TaskCoordinator:
    def __init__(self, agent_id):
        self.agent_id = agent_id
    
    def claim_task(self, task_id):
        """Claim a task by posting a claim message"""
        core.post(
            message=f"Agent {self.agent_id} claiming task {task_id}",
            subject=f"CLAIM:{task_id}",
            author=self.agent_id
        )
    
    def complete_task(self, task_id, result):
        """Mark task as completed with results"""
        core.post(
            message=f"Task {task_id} completed. Result: {result}",
            subject=f"COMPLETE:{task_id}",
            author=self.agent_id
        )
    
    def report_error(self, task_id, error):
        """Report task error"""
        core.post(
            message=f"Task {task_id} failed. Error: {error}",
            subject=f"ERROR:{task_id}",
            author=self.agent_id
        )

# Example usage
coordinator = TaskCoordinator("agent-worker-1")
coordinator.claim_task("001")
coordinator.complete_task("001", "Analysis completed: BTC trend is bullish")

In [ ]:
# Pattern 2: Heartbeat and Health Monitoring
class HealthMonitor:
    def __init__(self, agent_id):
        self.agent_id = agent_id
        self.last_heartbeat = datetime.now()
    
    def send_heartbeat(self, status_info=None):
        """Send heartbeat signal"""
        info = status_info or "healthy"
        core.post(
            message=f"Heartbeat from {self.agent_id}: {info}",
            subject=f"HEARTBEAT:{self.agent_id}",
            author=self.agent_id
        )
        self.last_heartbeat = datetime.now()
    
    def check_agent_health(self, max_silence_minutes=5):
        """Check if other agents are healthy based on heartbeats"""
        cutoff_time = datetime.now() - timedelta(minutes=max_silence_minutes)
        recent_heartbeats = core.read(since=cutoff_time)
        
        agents_seen = set()
        for msg in recent_heartbeats:
            if msg.subject.startswith('HEARTBEAT:'):
                agent_id = msg.subject.split(':')[1]
                agents_seen.add(agent_id)
        
        return list(agents_seen)

# Example usage
health_monitor = HealthMonitor("health-checker")
health_monitor.send_heartbeat("CPU: 45%, Memory: 2.1GB")

active_agents = health_monitor.check_agent_health()
print(f"Active agents in last 5 minutes: {active_agents}")

In [ ]:
# Pattern 3: Configuration and Coordination Messages
class ConfigManager:
    def __init__(self, agent_id):
        self.agent_id = agent_id
    
    def broadcast_config_update(self, config_key, config_value):
        """Broadcast configuration update to all agents"""
        core.post(
            message=f"Config update: {config_key} = {config_value}",
            subject=f"CONFIG:UPDATE",
            author=self.agent_id
        )
    
    def request_pause(self, reason):
        """Request all agents to pause operations"""
        core.post(
            message=f"PAUSE requested by {self.agent_id}. Reason: {reason}",
            subject="CONTROL:PAUSE",
            author=self.agent_id
        )
    
    def request_resume(self):
        """Request all agents to resume operations"""
        core.post(
            message=f"RESUME requested by {self.agent_id}",
            subject="CONTROL:RESUME",
            author=self.agent_id
        )

# Example usage
config_mgr = ConfigManager("config-manager")
config_mgr.broadcast_config_update("max_trade_amount", "$1000")
config_mgr.request_pause("Market volatility detected")

print("Configuration and control messages sent")

## Summary

This notebook demonstrates:

1. **Basic BBS operations** - posting and reading messages
2. **Python API usage** - programmatic access to the bulletin board
3. **Trading integration** - safe demo of trading tools
4. **AI coordination patterns** - task queues, health monitoring, configuration management

### Key Patterns for AI Agent Coordination:

- **Task Queue**: Use `TASK:` prefix for job posting, `CLAIM:` for claiming, `COMPLETE:` for results
- **Status Updates**: Use `STATUS:` prefix for agent state, `HEARTBEAT:` for health checks
- **Control Messages**: Use `CONTROL:` prefix for system-wide commands
- **Configuration**: Use `CONFIG:` prefix for setting updates

These patterns enable robust multi-agent coordination with the BBS as a message bus.